<a href="https://colab.research.google.com/github/JimMiller-0/Might-be-my-Year/blob/main/Might_be_my_Year.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting Started

This is a notebook for retreiving Fantasy Football Data from ESPN for analysis and predictive capabilities

## Install Necessary Librarys & SDKs

In [ ]:
# Install necessary SDKs

!pip install google-cloud-secret-manager
!pip install google-cloud-bigquery

#Extract Data from ESPN private Fantasy Football League

## Authenticate to ESPN Fantasy Football API

You'll need a couple things in order to authenticate to the ESPN Fantasy Football API. first retrieve and set league variables like league ID and season. Next you'll have to go through the process of getting the ESPN s2 and SWID access tokens. These access tokens we'll want to keep safe in GCP secrets manager.


Follow these instructions to get espn_s2 and swid: https://github.com/cwendt94/espn-api/discussions/150 or this blog: https://jman4190.medium.com/how-to-use-python-with-the-espn-fantasy-draft-api-ecde38621b1b


Follow these instructions to create secrets in GCP Secrets Manager: https://cloud.google.com/secret-manager/docs/creating-and-accessing-secrets#secretmanager-create-secret-console

Once the variables are set and the access tokens are secured in GCP secrets manager, go ahead and execute the code below. it will go retrieve the access tokens and set all the variable to use in each request going forward.

In [7]:
import pandas as pd
from google.cloud import secretmanager
import requests
import json
import time
import numpy as np
import datetime
import os


# You will need to get your league ID and ESPN S2 and SWID
# See https://github.com/cwendt94/espn-api/wiki/Football-Intro for details
# Recommended: Store SWID and ESPN S2 in a secrets manager, like gcp secrets manager: https://cloud.google.com/security/products/secret-manager

league_id = 1054374 # => set to league ID that you want to pull data from
season=2023 # => set to year you want to pull data from
url=f'https://lm-api-reads.fantasy.espn.com/apis/v3/games/ffl/seasons/{season}/segments/0/leagues/{league_id}?scoringPeriodId=17&view=mBoxscore&view=m' # => url of ESPN API. Note: this has changed over the years, navigate to the site and inspect network calls to get current endpoint

# Authenticate to Google Cloud
from google.colab import auth
auth.authenticate_user()

#Create a Client for secrets manager
client = secretmanager.SecretManagerServiceClient()
project_id = 'might-be-my-year' # => GCP Project ID where secrets manager is enabled
secret_espn_s2 = 'espn_s2' # name of secret in GCP secrets manager for espn_s2
secret_swid = 'swid' # name of secret in GCP secrets manager for swid

# Forge the paths to the latest version of the secrets with a F-string:
resource_name_espn_s2 = f"projects/{project_id}/secrets/{secret_espn_s2}/versions/latest"
resource_name_swid = f"projects/{project_id}/secrets/{secret_swid}/versions/latest"

# Load up the secrets to a variable at runtime:
response_espn_s2 = client.access_secret_version(name=resource_name_espn_s2)
response_swid = client.access_secret_version(name=resource_name_swid)

espn_s2 = response_espn_s2.payload.data.decode("UTF-8")
swid = response_swid.payload.data.decode("UTF-8")





## Get Data From ESPN API

We loop through a series of URLs representing the API endpoints for particular classes:


*   Draft
*   Free agency (weekly)
*   Players
*   Teams - Fantasy Football (not nfl)
*   Members
*   Player Weekly Box Score
*   Team Weekly Box Score
*   Matchups - Weekly Team vs Team Stats
*   League Settings

We save the raw data from the API to json file on our file system. these files aren't in a format to be loaded into BigQuery. In our next step we'll transform them so they are ready to be loaded into our BQ instance



In [8]:
# For each season we want to query the ESPN FF API and bring back the raw data and save to a json file with an apporpriate name (i.e draft_2023.json) {class_name}_{season}.json

for season in range(2018, 2024):
  base_url=f'https://lm-api-reads.fantasy.espn.com/apis/v3/games/ffl/'
  auth_cookies={"swid": swid, "espn_s2": espn_s2}

# we need to check the API route based on season/year. As the ESPN API updated over the years there were differences with how to interact with the API.
  if season >= 2018:
    endpoint = f'{base_url}seasons/{season}/segments/0/leagues/{league_id}'
    print(f'Getting data for year: {season}')
  else:
    ednppoint = f'{base_url}leagueHistory/{league_id}?seasonId={season}'
    print(f'Getting historical data for year: {season}')

# build the endpoint for each class, -> call the endpoint to get raw data -> sotre data in json file. Each class is a little different.


#Matchup - Weekly
  for week in range(1,18):

    matchup_filename = f"matchups_{season}_week_{week}.json"

    if os.path.exists(matchup_filename):
      os.remove(matchup_filename)
    else:
      pass

    r = requests.get(f'{endpoint}?view=mMatchup&view=mMatchupScore&scoringPeriodId={week}', cookies=auth_cookies)
    json_object = json.dumps(r.json(), indent = 4)
    with open(matchup_filename, "w") as outfile:
      outfile.write(json_object)
    time.sleep(0.08)

#Activity - Weekly - gets add/drops with bid amount for successful and failed bids along with what the team had left with in their FA budget. Also Trades- probably can be cleaned up but whatever
  for week in range(1,18):

    transactions_filename = f"transactions_{season}_week_{week}.json"

    if os.path.exists(transactions_filename):
      os.remove(transactions_filename)
    else:
      pass

    r = requests.get(f'{endpoint}?scoringPeriodId={week}&view=mDraftDetail&view=mStatus&view=mSettings&view=mTeam&view=mTransactions2&view=modular&view=mNav', cookies=auth_cookies)
    json_object = json.dumps(r.json(), indent = 4)
    with open(transactions_filename, "w") as outfile:
      outfile.write(json_object)
    time.sleep(0.08)

#Rosters -weekly
  for week in range(1,18):

    roster_filename = f"roster_{season}_week_{week}.json"

    if os.path.exists(roster_filename):
      os.remove(roster_filename)
    else:
      pass

    r = requests.get(f'{endpoint}?scoringPeriodId={week}&view=mRoster&view=mTeam', cookies=auth_cookies)
    json_object = json.dumps(r.json(), indent = 4)
    with open(roster_filename, "w") as outfile:
      outfile.write(json_object)
    time.sleep(0.08)

#NFL Schedule
  nfl_schedule_filename = f"nfl_schedule_{season}.json"

  if os.path.exists(nfl_schedule_filename):
    os.remove(nfl_schedule_filename)
  else:
    pass
  params = {'view': 'proTeamSchedules_wl'}
  r = requests.get(f'{base_url}seasons/{season}', params=params, cookies=auth_cookies)
  json_object = json.dumps(r.json(), indent = 4)
  with open(nfl_schedule_filename, "w") as outfile:
    outfile.write(json_object)
  time.sleep(0.08)

#Standings At End of Season
  standings_filename = f"standings_{season}.json"

  if os.path.exists(standings_filename):
    os.remove(standings_filename )
  else:
    pass
  r = requests.get(f'{endpoint}?view=mStandings', cookies=auth_cookies)
  json_object = json.dumps(r.json(), indent = 4)
  with open(standings_filename , "w") as outfile:
     outfile.write(json_object)
  time.sleep(0.08)

# Player_card
  player_card_filename = f"player_card_{season}.json"

  if os.path.exists(player_card_filename):
    os.remove(player_card_filename)
  else:
    pass
  data ={'players': {'filterStatsForTopScoringPeriodIds':{'value': 16}}}
  json_string = json.dumps(data)
  params = {'view': 'kona_playercard'}
  headers = {'x-fantasy-filter': json_string}
  r = requests.get(endpoint, params=params, headers=headers, cookies=auth_cookies)
  json_object = json.dumps(r.json(), indent = 4)
  with open(player_card_filename, "w") as outfile:
    outfile.write(json_object)
  time.sleep(0.08)

# League info
  league_filename = f"league_{season}.json"

  if os.path.exists(league_filename):
    os.remove(league_filename)
  else:
    pass
  r = requests.get(f'{endpoint}?view=mSettings', cookies=auth_cookies)
  json_object = json.dumps(r.json(), indent = 4)
  with open(league_filename, "w") as outfile:
     outfile.write(json_object)
  time.sleep(0.08)

# Teams - Fantasy Football (not nfl)
  teams_filename = f"teams_{season}.json"

  if os.path.exists(teams_filename):
    os.remove(teams_filename)
  else:
    pass
  r = requests.get(f'{endpoint}?&view=mTeam', cookies=auth_cookies)
  json_object = json.dumps(r.json(), indent = 4)
  with open(teams_filename, "w") as outfile:
    outfile.write(json_object)
  time.sleep(0.08)

# Players
  player_filename = f"players_{season}.json"

  if os.path.exists(player_filename):
    os.remove(player_filename)
  else:
    pass
  data = {'filterActive': {'value': True}}
  json_string = json.dumps(data)
  params = {'view': 'players_wl'}
  headers = {'x-fantasy-filter': json_string}
  r = requests.get(f'{endpoint}/players', params=params, headers=headers, cookies=auth_cookies)
  json_object = json.dumps(r.json(), indent = 4)
  with open(player_filename, "w") as outfile:
    outfile.write(json_object)
  time.sleep(0.08)

#Draft
  draft_filename = f"draft_{season}.json"

  if os.path.exists(draft_filename):
    os.remove(draft_filename)
  else:
    pass
  r = requests.get(f"{endpoint}?&view=mDraftDetail", cookies=auth_cookies)
  json_object = json.dumps(r.json(), indent = 4)
  with open(draft_filename, "w") as outfile:
    outfile.write(json_object)
  time.sleep(0.08)

# Free Agency (weekly)
  data = {'players': {'filterStatus': {'value': ['FREEAGENT', 'WAIVERS']}, 'limit': 2000, 'sortPercOwned': {'sortAsc': False, 'sortPriority': 1}}}
  json_string = json.dumps(data)
  headers = {'x-fantasy-filter': json_string}
  for week in range(1,18):

    fa_filename = f"free_agency_{season}_week_{week}.json"

    if os.path.exists(fa_filename):
      os.remove(fa_filename)
    else:
      pass

    r = requests.get(f'{endpoint}?scoringPeriodId={week}&view=kona_player_info', headers=headers, cookies=auth_cookies)
    json_object = json.dumps(r.json(), indent = 4)
    with open(fa_filename, "w") as outfile:
      outfile.write(json_object)
    time.sleep(0.08)




Getting data for year: 2018
Getting data for year: 2019
Getting data for year: 2020
Getting data for year: 2021
Getting data for year: 2022
Getting data for year: 2023


## Store Raw JSON Files in GCP Storage Bucket


This step isn't required, but can be helpful to store raw data in a known good state before applying all the transformations. We'll have to do a lot of work to make the raw json human readable... or at least fantasy football addict readable. This is a good way to reduce risk of having to repeat the whole extract if we mess up the files during the transforms.

In [ ]:

from google.cloud import storage
import uuid

project_id = 'might-be-my-year'  # Replace with your actual project ID

# Instantiates a client
storage_client = storage.Client(project=project_id)
random_uuid=str(uuid.uuid4())

# The name for the new bucket
bucket_name = f'fantasy-football-{league_id}-{random_uuid}'  # Replace with your bucket name

# Creates the new bucket
bucket = storage_client.create_bucket(bucket_name) # Call the create_bucket method to actually create the bucket
print(f"Bucket {bucket_name} created.") # Add a confirmation message

# Creates the new bucket
bucket = storage_client.bucket(bucket_name)

# Uploads files to the bucket
for filename in os.listdir():
    if filename.endswith(".json"):
        blob = bucket.blob(filename)
        blob.upload_from_filename(filename)
        print(f"File {filename} uploaded to {bucket_name}.")


# Transform Raw Data into Format and Schema for BigQuery

## Flatten File Format

In [ ]:
# test to make sure pandas can put the json into a dataframe
df = pd.json_normalize(data, record_path=['teams'])
df

## Convert to Parquet

In [ ]:
# Get Draft Details

draft_url= f'https://lm-api-reads.fantasy.espn.com/apis/v3/games/ffl/seasons/{season}/segments/0/leagues/{league_id}?view=mDraftDetail&view=mSettings&view=mTeam&view=modular&view=mNav'

r = requests.get(draft_url, cookies={'swid': swid, 'espn_s2': espn_s2})
if r.status_code == 200:

  draft_data = r.json()
  draft_data

# Save the data to a JSON file
  with open('draft_data.json', 'w') as f:
        json.dump(draft_data, f, indent=0)  # Use indent for pretty printing
  print('JSON file saved successfully.')
else:
  print('Request failed with status code:', r.status_code)

  # Read the JSON file
with open('draft_data.json', 'r') as f:
    draft_data = json.load(f)

draft_picks_df = pd.json_normalize(draft_data['draftDetail'], record_path=['picks'])
draft_members_df = pd.json_normalize(draft_data, record_path=['members'])
draft_settings_df = pd.json_normalize(draft_data['settings'])
draft_status_df = pd.json_normalize(draft_data['status'])
draft_teams_df = pd.json_normalize(draft_data, record_path=['teams'])

# test to make sure dataframes are working
draft_teams_df


## Store Parquet Files in GCP Storage Bucket

Same process as before. Good checkpoint to save known good state.


# Load Data into BigQuery for Storage & Analysis

Load Data in BigQuery - This makes is easier to manually slice and dice

In [6]:
from google.cloud import bigquery
import pyarrow as pa
from google.cloud.exceptions import NotFound
import pyarrow.parquet as pq

# set project id again
project_id = 'might-be-my-year'  # Replace with your actual project ID

# Construct a BigQuery client object.
bq_client = bigquery.Client(project=project_id)

# TODO(developer): Set dataset_id to the ID of the dataset to create.
dataset_id = "{}.fantasy_football".format(bq_client.project)

# Construct a full Dataset object to send to the API.
dataset = bigquery.Dataset(dataset_id)

#Specify the geographic location where the dataset should reside.
dataset.location = "US"

# Check if the dataset exists
try:
    bq_client.get_dataset(dataset_id)  # Make an API request.
    print(f"Dataset {dataset_id} already exists.")
except NotFound:
    print(f"Dataset {dataset_id} does not exist. Creating...")
# Send the dataset to the API for creation, with an explicit timeout.
# Raises google.api_core.exceptions.Conflict if the Dataset already
# exists within the project.
    dataset = bq_client.create_dataset(dataset, timeout=30)  # Make an API request.
    print("Created dataset {}.{}".format(bq_client.project, dataset.dataset_id))

# create lists of all dataframes with coooresponding table names
all_dfs = [draft_picks_df, draft_members_df, draft_settings_df, draft_status_df, draft_teams_df]
table_names = ['draft_picks', 'draft_members', 'draft_settings', 'draft_status', 'draft_teams']

for df, table_name in zip(all_dfs, table_names):

# Replace periods in column names with underscores
  df.columns = df.columns.str.replace('.', '_')  # Replace '.' with '_' in column names because thats what BQ needs
# Convert the DataFrame to a Parquet file
  table = pa.Table.from_pandas(df)
  pq.write_table(table, f'{table_name}.parquet')

# TODO(developer): Set table_id to the ID of the table to create.
  table_id = f'{dataset_id}.{table_name}_{season}'

  job_config = bigquery.LoadJobConfig(
      autodetect=True, source_format=bigquery.SourceFormat.PARQUET
  )

# open parquet file -> load the table from parquet file into bq table
  with open(f'{table_name}.parquet', "rb") as source_file:
      job = bq_client.load_table_from_file(
          source_file, table_id, job_config=job_config
      )

# Wait for the job to complete
  job.result()

  print("Loaded {} rows and {} columns to {}".format(job.output_rows, len(df.columns), table_id))

Dataset might-be-my-year.fantasy_football already exists.
Loaded 180 rows and 14 columns to might-be-my-year.fantasy_football.draft_picks_2023
Loaded 12 rows and 7 columns to might-be-my-year.fantasy_football.draft_members_2023
Loaded 1 rows and 131 columns to might-be-my-year.fantasy_football.draft_settings_2023
Loaded 1 rows and 56 columns to might-be-my-year.fantasy_football.draft_status_2023
Loaded 12 rows and 125 columns to might-be-my-year.fantasy_football.draft_teams_2023


# Analysis & Modeling

Analyze Data

Goal: Plot "Winners" and "Losers" based off of draft day auction price vs. production.

should be somthing like avg auction value vs price paid for on draft day vs inferred auction value based on end of year performance

plot 1: avg auction value vs price paid for on draft day. + difference shows league values that player/position more, - difference league values that player/position less

plot 2: avg auction value vs inferred auction value based on end of the year performance. + difference = player outperfomed expectations, - difference = player underperformed expectations

Plot 3: differnece in plot 1 vs difference in plot 2. quadrant plot: q1 =  players who under performed and the league overvalued. q2 = players who out performed and the league over valued. q3 = players who outperformed and the league undervalued. q4 =  players that underperformed and the league undervalued


In [ ]:
#